*   Author: Zhuoning Yuan
*   Project: https://github.com/yzhuoning/LibAUC


# **Installing LibAUC**

In [ ]:
!pip install libauc

# **Importing AUC Training Pipeline**

In [ ]:
from libauc.losses import AUCMLoss
from libauc.optimizers import PESG
from libauc.models import ResNet20
from libauc.datasets import CIFAR10
from libauc.datasets import imbalance_generator 

import torch 
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from sklearn.metrics import roc_auc_score

class ImageDataset(Dataset):
    def __init__(self, images, targets, image_size=32, crop_size=30, mode='train'):
       self.images = images.astype(np.uint8)
       self.targets = targets
       self.mode = mode
       self.transform_train = transforms.Compose([                                                
                              transforms.ToTensor(),
                              transforms.RandomCrop((crop_size, crop_size), padding=None),
                              transforms.RandomHorizontalFlip(),
                              transforms.Resize((image_size, image_size)),
                              ])
       self.transform_test = transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Resize((image_size, image_size)),
                              ])
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        target = self.targets[idx]
        image = Image.fromarray(image.astype('uint8'))
        if self.mode == 'train':
            image = self.transform_train(image)
        else:
            image = self.transform_test(image)
        return image, target

# paramaters
SEED = 123
BATCH_SIZE = 128
imratio = 0.1
lr = 0.1
gamma = 500
weight_decay = 1e-4
margin = 1.0


# dataloader 
(train_data, train_label), (test_data, test_label) = CIFAR10()
(train_images, train_labels) = imbalance_generator(train_data, train_label, imratio=imratio, shuffle=True, random_seed=SEED)
(test_images, test_labels) = imbalance_generator(test_data, test_label, is_balanced=True,  random_seed=SEED)

trainloader = torch.utils.data.DataLoader(ImageDataset(train_images, train_labels), batch_size=BATCH_SIZE, shuffle=True, num_workers=1, pin_memory=True, drop_last=True)
testloader = torch.utils.data.DataLoader( ImageDataset(test_images, test_labels, mode='test'), batch_size=BATCH_SIZE, shuffle=False, num_workers=1,  pin_memory=True)

# model 
model = ResNet20(pretrained=False, num_classes=1)
model = model.cuda()

# loss & optimizer
Loss = AUCMLoss(imratio=imratio)
optimizer = PESG(model, 
                 a=Loss.a, 
                 b=Loss.b, 
                 alpha=Loss.alpha, 
                 imratio=imratio, 
                 lr=lr, 
                 gamma=gamma, 
                 margin=margin, 
                 weight_decay=weight_decay)

NUM_SAMPLES: [27777], POS:NEG: [2777 : 25000], POS_RATIO: 0.1000
NUM_SAMPLES: [10000], POS:NEG: [5000 : 5000], POS_RATIO: 0.5000


# **Pytorch Learning Rate Scheduling**
*   CosineAnnealingLR
*   ReduceLROnPlateau
*   MultiStepLR



Reference: https://pytorch.org/docs/stable/optim.html


In [ ]:
def reset_model():
    # loss & optimizer
    Loss = AUCMLoss(imratio=imratio)
    optimizer = PESG(model, 
                    a=Loss.a, 
                    b=Loss.b, 
                    alpha=Loss.alpha, 
                    imratio=imratio, 
                    lr=lr, 
                    gamma=gamma, 
                    margin=margin, 
                    weight_decay=weight_decay)
    return Loss, optimizer

### CosineAnnealingLR

In [ ]:
total_epochs = 10
Loss, optimizer = reset_model()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(trainloader)*total_epochs)

In [ ]:
model.train()    
for epoch in range(total_epochs):
     for i, (data, targets) in enumerate(trainloader):
         data, targets  = data.cuda(), targets.cuda()
         y_pred = model(data)
         loss = Loss(y_pred, targets)
         optimizer.zero_grad()
         loss.backward()
         optimizer.step()
         scheduler.step()
     print("epoch: {}, loss: {:4f}, lr:{:4f}".format(epoch, loss.item(), optimizer.lr))          

epoch: 0, train_loss: 0.089644, lr:0.097575
epoch: 1, train_loss: 0.071552, lr:0.090493
epoch: 2, train_loss: 0.074371, lr:0.079448
epoch: 3, train_loss: 0.067860, lr:0.065520
epoch: 4, train_loss: 0.070255, lr:0.050072
epoch: 5, train_loss: 0.024033, lr:0.034618
epoch: 6, train_loss: 0.053591, lr:0.020669
epoch: 7, train_loss: 0.076163, lr:0.009592
epoch: 8, train_loss: 0.062752, lr:0.002470
epoch: 9, train_loss: 0.063125, lr:0.000000


### ReduceLROnPlateau

In [ ]:
total_epochs = 20
Loss, optimizer = reset_model()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                       patience=3,  
                                                       verbose=True, 
                                                       factor=0.5, 
                                                       threshold=0.001,
                                                       min_lr=0.00001)

In [ ]:
model.train()    
for epoch in range(total_epochs):
     for i, (data, targets) in enumerate(trainloader):
         data, targets  = data.cuda(), targets.cuda()
         y_pred = model(data)
         loss = Loss(y_pred, targets)
         optimizer.zero_grad()
         loss.backward()
         optimizer.step()
     scheduler.step(loss)
     print("epoch: {}, loss: {:4f}, lr:{:4f}".format(epoch, loss.item(), optimizer.lr))          

epoch: 0, loss: 0.063075, lr:0.100000
epoch: 1, loss: 0.106665, lr:0.100000
epoch: 2, loss: 0.069563, lr:0.100000
epoch: 3, loss: 0.043283, lr:0.100000
epoch: 4, loss: 0.107795, lr:0.100000
epoch: 5, loss: 0.084268, lr:0.100000
epoch: 6, loss: 0.077447, lr:0.100000
Epoch     8: reducing learning rate of group 0 to 5.0000e-02.
epoch: 7, loss: 0.044456, lr:0.100000
epoch: 8, loss: 0.038239, lr:0.050000
epoch: 9, loss: 0.049636, lr:0.050000
epoch: 10, loss: 0.036171, lr:0.050000
epoch: 11, loss: 0.019520, lr:0.050000
epoch: 12, loss: 0.037929, lr:0.050000
epoch: 13, loss: 0.067369, lr:0.050000
epoch: 14, loss: 0.040822, lr:0.050000
Epoch    16: reducing learning rate of group 0 to 2.5000e-02.
epoch: 15, loss: 0.101739, lr:0.050000
epoch: 16, loss: 0.023114, lr:0.025000
epoch: 17, loss: 0.000277, lr:0.025000
epoch: 18, loss: 0.041810, lr:0.025000
epoch: 19, loss: 0.000383, lr:0.025000


### MultiStepLR

In [ ]:
total_epochs = 20
Loss, optimizer = reset_model()
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10,15], gamma=0.1)

In [ ]:
# reset model
model.train()    
for epoch in range(total_epochs):
     for i, (data, targets) in enumerate(trainloader):
         data, targets  = data.cuda(), targets.cuda()
         y_pred = model(data)
         loss = Loss(y_pred, targets)
         optimizer.zero_grad()
         loss.backward()
         optimizer.step()
     scheduler.step()
     print("epoch: {}, loss: {:4f}, lr:{:4f}".format(epoch, loss.item(), optimizer.lr))          

epoch: 0, loss: 0.087777, lr:0.100000
epoch: 1, loss: 0.069161, lr:0.100000
epoch: 2, loss: 0.074617, lr:0.100000
epoch: 3, loss: 0.101814, lr:0.100000
epoch: 4, loss: 0.063635, lr:0.100000
epoch: 5, loss: 0.013147, lr:0.100000
epoch: 6, loss: 0.043657, lr:0.100000
epoch: 7, loss: 0.048249, lr:0.100000
epoch: 8, loss: 0.067078, lr:0.100000
epoch: 9, loss: 0.025596, lr:0.100000
epoch: 10, loss: 0.021931, lr:0.010000
epoch: 11, loss: 0.016166, lr:0.010000
epoch: 12, loss: 0.064753, lr:0.010000
epoch: 13, loss: 0.076789, lr:0.010000
epoch: 14, loss: 0.035034, lr:0.010000
epoch: 15, loss: 0.048091, lr:0.001000
epoch: 16, loss: 0.064210, lr:0.001000
epoch: 17, loss: 0.033093, lr:0.001000
epoch: 18, loss: 0.079196, lr:0.001000
epoch: 19, loss: 0.034892, lr:0.001000
